In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls /content/drive/My\ Drive/Colab\ Notebooks

 558_Assignments_Code  'Fondation Final Project'   my_own_code
 data		        Kaggle			  'powerful word.ipynb'


In [3]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
mt = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/medicaltranscriptions.csv')
icd = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/allvalid2011 (detailed titles headings).xlsx')

In [0]:
mt

In [5]:
# word2vec pretrained model
model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/PubMed-and-PMC-w2v.bin", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# medical transcript

In [0]:
# Generate Vec for each row
description_vec_mt=np.empty((0, 200), float)

stop_words = set(stopwords.words('english'))
for i in range(len(mt['description'])):
    word2vec_ls = np.empty((0,200),dtype=float)
    #tokenize each sentence & remove stop words
    word_tokens = nltk.word_tokenize(mt['description'][i])
    word_tokens = [w.lower() for w in word_tokens] 
    word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
    word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
    #pos tag & remove useless words & document representation
    mt_token_pos = nltk.pos_tag(word_tokens)
    for word, pos in mt_token_pos:
        if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
            try:
                word2vec_ls = np.vstack([word2vec_ls, model.word_vec(word)])
            except:
                pass
    if np.any(np.isnan(word2vec_ls)) or len(word2vec_ls)==0:
        desc_vec = np.zeros((1,200))
    else:
        desc_vec = np.mean(word2vec_ls, axis=0).reshape(1,200)
    description_vec_mt = np.vstack([description_vec_mt, desc_vec])

In [7]:
description_vec_mt

array([[-0.21089798,  0.17600803,  0.141195  , ...,  0.02037348,
        -0.12230716, -0.3454161 ],
       [-0.35882969,  0.14851366,  0.03550289, ..., -0.07603217,
         0.05409601, -0.15183919],
       [-0.35882969,  0.14851366,  0.03550289, ..., -0.07603217,
         0.05409601, -0.15183919],
       ...,
       [-0.14429586,  0.28950928,  0.00662719, ..., -0.04772653,
        -0.30058188, -0.19395648],
       [-0.10533292, -0.0736654 ,  0.01838912, ...,  0.28866098,
        -0.17113749, -0.1145479 ],
       [-0.16375436,  0.12372051,  0.0399576 , ...,  0.23361799,
        -0.00494082, -0.0747135 ]])

# ICD Title

In [0]:
icd

In [0]:
#
import collections
code_title_dic = collections.defaultdict(list)
icd_titles = icd['ICD Title'].values.tolist()
icd_codes = icd['Code'].values.tolist()

main_code = None

for code,des in zip(icd_codes,icd_titles):
    if '-' in code:
        continue
    elif '.' in code and main_code in code:
        code_title_dic[main_code].append(des)
    else:
        main_code = code
        code_title_dic[main_code].append(des)

In [12]:
code_title_dic['A02']

['Other salmonella infections',
 'Salmonella enteritis',
 'Salmonella septicemia',
 'Localized salmonella infections',
 'Other specified salmonella infections',
 'Salmonella infection, unspecified']

In [0]:
for k, v in code_title_dic.items():
  s = ''
  for each in code_title_dic[k]:
    s+=each+' '
  word_tokens = nltk.word_tokenize(s)
  word_tokens = [w.lower() for w in word_tokens] 
  word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
  word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
  word_tokens = list(set(word_tokens))
  mt_token_pos = nltk.pos_tag(word_tokens)
  word_tokens=[]
  for word, pos in mt_token_pos:
      if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
        word_tokens.append(word)
  code_title_dic[k] = word_tokens

In [0]:
'''s = ''
for each in code_title_dic['A02']:
  s+=each+' '
word_tokens = nltk.word_tokenize(s)
word_tokens = [w.lower() for w in word_tokens] 
word_tokens = [w.lower() for w in word_tokens if not w in stop_words] 
word_tokens = [w.lower() for w in word_tokens if w.isalpha()]
word_tokens = list(set(word_tokens))
mt_token_pos = nltk.pos_tag(word_tokens)
word_tokens=[]
for word, pos in mt_token_pos:
    if (pos=='NN' or pos=='NNP' or pos=='NNS' or pos=='NNPS'):
      word_tokens.append(word)'''

In [28]:
code_title_dic['A02']

['infection', 'infections', 'salmonella', 'septicemia']

In [0]:
#generate vector for each medical case
description_vec_icd = np.empty((0, 200), float)
for k,v in code_title_dic.items():
  for word in code_title_dic[k]:
    word2vec_ls = np.empty((0,200),dtype=float)
    try:
      word2vec_ls = np.vstack([word2vec_ls, model.word_vec(word)])
    except:
      pass
  if np.any(np.isnan(word2vec_ls)) or len(word2vec_ls)==0:
      desc_vec = np.zeros((1,200))
  else:
      desc_vec = np.mean(word2vec_ls, axis=0).reshape(1,200)
  description_vec_icd = np.vstack([description_vec_icd, desc_vec])

In [16]:
len(code_title_dic)

1855

In [17]:
len(description_vec_icd)

1855

In [18]:
description_vec_icd

array([[ 0.08815501, -0.05478497,  0.06878097, ..., -0.07263169,
        -0.12873437,  0.08623968],
       [ 0.03250572,  0.03838501,  0.0490371 , ..., -0.04861115,
        -0.1156738 , -0.01287178],
       [-0.11993292,  0.07084431,  0.13983248, ..., -0.06663563,
        -0.06310885, -0.09680044],
       ...,
       [-0.15487147,  0.62171114,  0.01705617, ..., -0.30654355,
        -0.48337165,  0.01517189],
       [ 0.23919172,  0.06692002,  0.09771204, ..., -0.22375459,
        -0.13577058,  0.07225289],
       [ 0.04660308,  0.13164124,  0.10574624, ..., -0.11507406,
        -0.16049868, -0.0474148 ]])

In [0]:
code_vec_dic = collections.defaultdict(list)
icd_codes = list(code_title_dic.keys())
icd_vec = description_vec_icd

main_code = None

for code,des in zip(icd_codes, icd_vec):
  main_code = code
  code_vec_dic[main_code].append(des)

In [35]:
code_vec_dic['A01']

[array([-0.09208631,  0.83537179,  0.09508243, -0.12652601, -0.21098493,
        -0.15395205, -0.33225271, -0.01164279,  0.05875952,  0.4471848 ,
         0.22602338, -0.23362112, -0.04224472, -0.22180896, -0.52228004,
        -0.00710988,  0.08741832,  0.0212653 ,  0.40927649, -0.02913291,
         0.50262928, -0.3635855 ,  0.4475494 ,  0.46390885,  0.69855571,
         0.13422312,  0.4426927 ,  0.13922253,  0.06152431,  0.09413553,
         0.23957419,  0.44637954,  0.24685931,  0.5619418 , -0.32942134,
         0.13322391, -0.23679824, -0.12797232, -0.13951226,  0.06674336,
         0.0418866 , -0.07084281, -0.81858093, -0.12338661,  0.05207586,
        -0.38203821, -0.27468768,  0.28239301, -0.30298561, -0.14415789,
         0.09946578, -0.08578627, -0.25652975,  0.2881825 , -0.23683785,
        -0.25296435, -0.1544238 , -0.41246545,  0.22654651,  0.0113169 ,
         0.20139533, -0.01240507,  0.03181112, -0.2431208 ,  0.22367685,
         0.31246856,  0.5097028 ,  0.39152038,  0.3

# compare similarity

In [36]:
len(code_vec_dic)

1855

In [37]:
# code_vec_dic.values represents icd vector representation
# description_vec_mt represents medical transcript vector representation

#build similarity matrix.
similarity_matrix = pd.DataFrame(np.zeros((4999, len(code_vec_dic)), dtype=float), columns=icd_codes)
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y48,Y49,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(4999):
  print('Calculating {}th row'.format(i+1))
  for j in similarity_matrix.columns:
    similarity_matrix.loc[i,j] = cosine_similarity(description_vec_mt[i].reshape(1,-1), code_vec_dic[j][0].reshape(1,-1))

Calculating 1th row
Calculating 2th row
Calculating 3th row
Calculating 4th row
Calculating 5th row
Calculating 6th row
Calculating 7th row
Calculating 8th row
Calculating 9th row
Calculating 10th row
Calculating 11th row
Calculating 12th row
Calculating 13th row
Calculating 14th row
Calculating 15th row
Calculating 16th row
Calculating 17th row
Calculating 18th row
Calculating 19th row
Calculating 20th row
Calculating 21th row
Calculating 22th row
Calculating 23th row
Calculating 24th row
Calculating 25th row
Calculating 26th row
Calculating 27th row
Calculating 28th row
Calculating 29th row
Calculating 30th row
Calculating 31th row
Calculating 32th row
Calculating 33th row
Calculating 34th row
Calculating 35th row
Calculating 36th row
Calculating 37th row
Calculating 38th row
Calculating 39th row
Calculating 40th row
Calculating 41th row
Calculating 42th row
Calculating 43th row
Calculating 44th row
Calculating 45th row
Calculating 46th row
Calculating 47th row
Calculating 48th row
C

In [0]:
#similarity_matrix = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/medical data/similarity_matrix.csv')

In [40]:
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y48,Y49,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89
0,0.008031,0.009162,0.030430,0.008716,0.069628,0.061562,0.027367,0.029964,0.012408,0.098596,0.105524,0.021422,0.023782,0.005626,-0.000017,0.131033,-0.005137,0.018063,0.017144,0.013671,-0.006110,0.035555,-0.004478,0.093721,0.086473,0.030430,0.000323,0.022942,0.022942,0.128461,0.012015,-0.025293,0.044715,0.055187,0.037924,0.029145,0.015803,0.015803,0.022942,0.010023,...,0.018063,0.018797,0.000853,-0.007081,0.016949,0.016693,0.033864,0.024131,0.012654,0.028013,0.000959,0.031622,0.083765,0.078749,0.075831,0.052237,0.035548,0.053220,-0.000220,-0.000220,0.022344,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.054756,0.055202,0.010282,0.022942,-0.013653,0.004723,0.045799
1,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.025691,0.022470,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804
2,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.025691,0.022470,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804
3,0.086435,0.042906,-0.000766,0.004666,0.059443,0.066358,0.015444,0.005131,0.014412,0.098106,0.102915,0.014972,0.052180,0.011748,0.057248,0.003914,0.017220,-0.000809,-0.001108,-0.002217,-0.011915,-0.001977,0.009755,0.075597,0.059914,-0.000766,0.028230,0.000819,0.000819,0.003385,-0.018897,0.133358,0.043091,0.022772,0.011875,-0.000480,-0.001540,-0.001540,0.000819,0.076397,...,-0.000809,0.002822,-0.006053,0.002634,0.014191,0.005229,0.020781,0.011877,-0.002536,0.001669,0.072805,0.014309,0.101470,0.091801,0.101447,0.081573,0.021088,0.104238,0.065235,0.065235,-0.000624,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,0.074786,0.045540,0.035142,0.000819,0.077004,0.060869,0.054725
4,0.009404,-0.004979,0.004491,-0.024947,0.021792,0.010741,0.006847,0.004693,0.016515,0.038717,0.034960,-0.039169,0.005296,-0.001531,-0.005549,-0.023470,-0.049380,0.008265,0.007579,0.005001,-0.026718,-0.001382,-0.008696,-0.013136,0.019019,0.004491,-0.057787,0.011942,0.011942,-0.024083,0.012253,0.052458,-0.001458,-0.027215,0.002958,0.005854,0.006581,0.006581,0.011942,0.003469,...,0.008265,-0.003247,-0.004312,-0.033122,-0.008846,-0.003545,-0.004613,-0.012119,0.004251,-0.000991,0.012784,0.013217,0.026571,0.022889,0.027760,-0.001091,0.013628,0.028687,-0.005166,-0.005166,0.005616,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.009556,-0.002561,0.030603,0.

In [41]:
max_index=[]
max_val = []
for i in range(4999):
  max_index.append(np.argmax(similarity_matrix.iloc[i], axis=1))
  max_val.append(similarity_matrix.loc[i,max_index[i]])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return bound(*args, **kwds)


In [0]:
similarity_matrix['max_index'] = max_index

In [0]:
similarity_matrix['max_prob'] = max_val

In [49]:
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob
0,0.008031,0.009162,0.030430,0.008716,0.069628,0.061562,0.027367,0.029964,0.012408,0.098596,0.105524,0.021422,0.023782,0.005626,-0.000017,0.131033,-0.005137,0.018063,0.017144,0.013671,-0.006110,0.035555,-0.004478,0.093721,0.086473,0.030430,0.000323,0.022942,0.022942,0.128461,0.012015,-0.025293,0.044715,0.055187,0.037924,0.029145,0.015803,0.015803,0.022942,0.010023,...,0.000853,-0.007081,0.016949,0.016693,0.033864,0.024131,0.012654,0.028013,0.000959,0.031622,0.083765,0.078749,0.075831,0.052237,0.035548,0.053220,-0.000220,-0.000220,0.022344,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.054756,0.055202,0.010282,0.022942,-0.013653,0.004723,0.045799,O12,0.170697
1,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804,F68,0.057977
2,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804,F68,0.057977
3,0.086435,0.042906,-0.000766,0.004666,0.059443,0.066358,0.015444,0.005131,0.014412,0.098106,0.102915,0.014972,0.052180,0.011748,0.057248,0.003914,0.017220,-0.000809,-0.001108,-0.002217,-0.011915,-0.001977,0.009755,0.075597,0.059914,-0.000766,0.028230,0.000819,0.000819,0.003385,-0.018897,0.133358,0.043091,0.022772,0.011875,-0.000480,-0.001540,-0.001540,0.000819,0.076397,...,-0.006053,0.002634,0.014191,0.005229,0.020781,0.011877,-0.002536,0.001669,0.072805,0.014309,0.101470,0.091801,0.101447,0.081573,0.021088,0.104238,0.065235,0.065235,-0.000624,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,0.074786,0.045540,0.035142,0.000819,0.077004,0.060869,0.054725,C30,0.136872
4,0.009404,-0.004979,0.004491,-0.024947,0.021792,0.010741,0.006847,0.004693,0.016515,0.038717,0.034960,-0.039169,0.005296,-0.001531,-0.005549,-0.023470,-0.049380,0.008265,0.007579,0.005001,-0.026718,-0.001382,-0.008696,-0.013136,0.019019,0.004491,-0.057787,0.011942,0.011942,-0.024083,0.012253,0.052458,-0.001458,-0.027215,0.002958,0.005854,0.006581,0.006581,0.011942,0.003469,...,-0.004312,-0.033122,-0.008846,-0.003545,-0.004613,-0.012119,0.004251,-0.000991,0.012784,0.013217,0.026571,0.022889,0.027760,-0.001091,0.013628,0.028687,-0.005166,-0.005166,0.005616,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.009556,-0.002561,0.030603,0.011942,0.030133,-0.002241,0.0

In [52]:
similarity_matrix['max_prob'].mean()

0.1694002981355477

In [0]:
exam_ones = similarity_matrix[similarity_matrix['max_prob']>0.4]

In [86]:
exam_ones[exam_ones.index == 4687]

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,max_prob
4687,0.225937,0.258877,0.201047,0.22229,0.251328,0.244785,0.174915,0.167114,0.242728,0.224466,0.232976,0.215627,0.188778,0.170582,0.087259,0.354916,0.151236,0.166493,0.167693,0.17165,0.199474,0.229185,0.16849,0.169104,0.293753,0.201047,0.138132,0.15896,0.15896,0.356286,0.176139,0.190909,0.234811,0.271652,0.214007,0.193894,0.169328,0.169328,0.15896,0.138432,...,0.17893,0.171687,0.196136,0.169859,0.189013,0.172058,0.172641,0.193211,0.212417,0.193499,0.21317,0.202201,0.194672,0.147127,0.207814,0.244102,0.054982,0.054982,0.181626,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.186711,0.30421,0.224099,0.323955,0.15896,0.45178,0.085301,0.197505,J62,0.556868


In [66]:
from sklearn.utils import shuffle
shuffle(exam_ones)[:10]['max_index']

4146    R96
4088    C06
2435    C06
1989    R11
4687    J62
2654    P03
49      R96
1681    T26
4766    K03
3460    I85
Name: max_index, dtype: object

In [77]:
mt[2654:2655].values

array([[2654,
        ' Ultrasound BPP - Advanced maternal age and hypertension.',
        ' Obstetrics / Gynecology', ' Biophysical Profile - 1 ',
        'HISTORY: , Advanced maternal age and hypertension.,FINDINGS:,  There is a single live intrauterine pregnancy with a vertex lie, posterior placenta, and adequate amniotic fluid.  The amniotic fluid index is 23.2 cm.  Estimated gestational age based on prior ultrasound is 36 weeks 4 four days with an estimated date of delivery of 03/28/08.  Based on fetal measurements obtained today, estimated fetal weight is 3249 plus or minus 396 g, 7 pounds 3 ounces plus or minus 14 ounces, which places the fetus in the 66th percentile for the estimated gestational age.  Fetal heart motion at a rate of 156 beats per minute is documented.  The cord Doppler ratio is normal at 2.2.  The biophysical profile score, assessing fetal breathing movement, gross body movement, fetal tone, and qualitative amniotic fluid volume is 8/8.,IMPRESSION:,1.  Single l

In [27]:
icdto27 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Fondation Final Project/data/map data/ICDto27.csv')
icdto27

,ICD,27
0,A16,1
1,A17,1
2,A18,1
3,A19,1
4,A50,2
...,...,...
2082,Y34,27
2083,Y35,27
2084,Y36,27
2085,Y87.2,27


In [0]:
dic = dict()
for i in range(len(icdto27)):
  dic[icdto27['ICD'][i]] = icdto27['27'][i]

In [36]:
dic['F68']

KeyError: ignored

In [37]:
similarity_matrix['Rankable_Cause'] = similarity_matrix['max_index'].map(dic){}

ValueError: ignored

In [35]:
similarity_matrix

,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,A30,A31,A32,A33,A34,A35,A36,A37,A38,A39,A40,A41,A42,A43,A44,A46,A48,...,Y50,Y51,Y52,Y53,Y54,Y55,Y56,Y57,Y58,Y59,Y60,Y61,Y62,Y63,Y64,Y65,Y66,Y69,Y70,Y71,Y72,Y73,Y74,Y75,Y76,Y77,Y78,Y79,Y80,Y81,Y82,Y83,Y84,Y85,Y86,Y87,Y88,Y89,max_index,Rankable_Cause
0,0.008031,0.009162,0.030430,0.008716,0.069628,0.061562,0.027367,0.029964,0.012408,0.098596,0.105524,0.021422,0.023782,0.005626,-0.000017,0.131033,-0.005137,0.018063,0.017144,0.013671,-0.006110,0.035555,-0.004478,0.093721,0.086473,0.030430,0.000323,0.022942,0.022942,0.128461,0.012015,-0.025293,0.044715,0.055187,0.037924,0.029145,0.015803,0.015803,0.022942,0.010023,...,0.000853,-0.007081,0.016949,0.016693,0.033864,0.024131,0.012654,0.028013,0.000959,0.031622,0.083765,0.078749,0.075831,0.052237,0.035548,0.053220,-0.000220,-0.000220,0.022344,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.022030,0.054756,0.055202,0.010282,0.022942,-0.013653,0.004723,0.045799,O12,17.0
1,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804,F68,NaN
2,0.024951,0.014334,0.027199,0.015120,0.039057,0.025654,0.028170,0.030069,0.022529,0.044352,0.048246,-0.019626,0.022233,0.017824,-0.001291,0.025403,-0.042313,0.025691,0.025416,0.024312,0.007524,0.026724,0.013030,0.048617,0.042757,0.027199,-0.054725,0.027032,0.027032,0.025240,0.029146,0.048043,0.021919,0.008005,0.025899,0.027234,0.025001,0.025001,0.027032,0.018638,...,0.019457,-0.003715,0.009498,0.011657,0.015749,0.009083,0.023971,0.027776,0.001285,0.030944,0.033105,0.031230,0.031754,0.018559,0.032390,0.021861,-0.006035,-0.006035,0.024624,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.023793,0.021669,0.033277,0.019324,0.027032,-0.014808,0.000192,0.029804,F68,NaN
3,0.086435,0.042906,-0.000766,0.004666,0.059443,0.066358,0.015444,0.005131,0.014412,0.098106,0.102915,0.014972,0.052180,0.011748,0.057248,0.003914,0.017220,-0.000809,-0.001108,-0.002217,-0.011915,-0.001977,0.009755,0.075597,0.059914,-0.000766,0.028230,0.000819,0.000819,0.003385,-0.018897,0.133358,0.043091,0.022772,0.011875,-0.000480,-0.001540,-0.001540,0.000819,0.076397,...,-0.006053,0.002634,0.014191,0.005229,0.020781,0.011877,-0.002536,0.001669,0.072805,0.014309,0.101470,0.091801,0.101447,0.081573,0.021088,0.104238,0.065235,0.065235,-0.000624,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,-0.001020,0.074786,0.045540,0.035142,0.000819,0.077004,0.060869,0.054725,C30,4.0
4,0.009404,-0.004979,0.004491,-0.024947,0.021792,0.010741,0.006847,0.004693,0.016515,0.038717,0.034960,-0.039169,0.005296,-0.001531,-0.005549,-0.023470,-0.049380,0.008265,0.007579,0.005001,-0.026718,-0.001382,-0.008696,-0.013136,0.019019,0.004491,-0.057787,0.011942,0.011942,-0.024083,0.012253,0.052458,-0.001458,-0.027215,0.002958,0.005854,0.006581,0.006581,0.011942,0.003469,...,-0.004312,-0.033122,-0.008846,-0.003545,-0.004613,-0.012119,0.004251,-0.000991,0.012784,0.013217,0.026571,0.022889,0.027760,-0.001091,0.013628,0.028687,-0.005166,-0.005166,0.005616,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.003824,0.009556,-0.002561,0.030603,0.011942,0.030133,-0.002241,0.018109,A38,22.